# Contextual Chunk Headers (CCH) in Simple RAG

Retrieval-Augmented Generation (RAG) improves the factual accuracy of language models by retrieving relevant external knowledge before generating a response. However, standard chunking often loses important context, making retrieval less effective.

Contextual Chunk Headers (CCH) enhance RAG by prepending high-level context (like document titles or section headers) to each chunk before embedding them. This improves retrieval quality and prevents out-of-context responses.

## Steps in this Notebook:

1. **Data Ingestion**: Load and preprocess the text data.
2. **Chunking with Contextual Headers**: Extract section titles and prepend them to chunks.
3. **Embedding Creation**: Convert context-enhanced chunks into numerical representations.
4. **Semantic Search**: Retrieve relevant chunks based on a user query.
5. **Response Generation**: Use a language model to generate a response from retrieved text.
6. **Evaluation**: Assess response accuracy using a scoring system.

### 简单RAG中的上下文块头（CCH）技术  

检索增强生成（RAG）通过在生成响应前检索相关外部知识，提升语言模型的事实准确性。然而，标准分块方法常丢失重要上下文，导致检索效率低下。  

**上下文块头（Contextual Chunk Headers, CCH）** 技术通过在嵌入前为每个文本块添加高层级上下文（如文档标题或章节标题）来增强RAG系统。这一方法可提升检索质量，并避免生成脱离上下文的响应。  


### 本笔记本的实现步骤：  
1. **数据摄入**：加载并预处理文本数据。  
2. **带上下文头的分块**：提取章节标题并将其添加到文本块前。  
3. **嵌入创建**：将上下文增强的文本块转换为数值表示。  
4. **语义搜索**：基于用户查询检索相关文本块。  
5. **响应生成**：使用语言模型从检索到的文本中生成回答。  
6. **评估**：使用评分系统评估响应的准确性。

### 简单RAG中的上下文块头（CCH）技术解析  

#### 一、CCH技术的核心目标与创新点  
检索增强生成（RAG）通过检索外部知识提升语言模型的事实准确性，但传统分块常丢失文档结构信息（如章节标题、段落关系），导致检索结果偏离上下文。**上下文块头（Contextual Chunk Headers, CCH）** 的核心创新是：**在每个文本块前添加高层级上下文（如文档标题、章节头）**，形成“标题+内容”的块结构，从而在嵌入和检索阶段保留文档语义层级。  


#### 二、CCH技术的工作原理  
1. **结构保留**：  
   将文档的层级信息（如“第三章 人工智能核心技术”）作为块头，与具体内容（如“机器学习是AI的基础...”）拼接成新块：  
   ```plaintext
   [块头] 第三章 人工智能核心技术  
   [块内容] 机器学习通过算法从数据中学习模式，分为监督学习、无监督学习...
   ```  

2. **语义增强**：  
   块头提供了内容的主题归属和上下文关系，使嵌入向量同时包含：  
   - 块内容的具体语义（如“机器学习”）  
   - 块的层级上下文（如“第三章”的主题相关性）  

3. **检索优化**：  
   当用户查询与块头主题匹配时（如“人工智能核心技术有哪些”），带块头的文本块会因更高的语义相关性被优先检索，减少跨主题误匹配。  


#### 三、Notebook实现步骤详解  

##### 1. 数据摄入与预处理  
```python
# 示例：从结构化文档中提取带层级的文本
def extract_text_with_headers(doc_path):
    """从包含标题和内容的文档中提取带层级的文本"""
    # 假设文档结构为：标题1 -> 子标题1.1 -> 内容 -> 子标题1.2 -> 内容...
    headers = []  # 存储各级标题
    content = []  # 存储内容段落
    # 实际实现需根据文档格式（如PDF、Markdown）解析结构
    return headers, content
```  
- **关键逻辑**：解析文档时保留标题层级，区分“章标题”“节标题”“子节标题”等不同粒度的上下文。  


##### 2. 带上下文头的分块  
```python
def chunk_with_context_headers(headers, content, chunk_size=1000, overlap=200):
    """将文本分块并添加对应的上下文头"""
    chunks = []
    current_header = ""
    
    # 遍历标题和内容，建立标题与内容的归属关系
    for i, (header, text) in enumerate(zip(headers, content)):
        if header:  # 遇到新标题时更新当前上下文头
            current_header = header
        
        # 分块处理内容，并添加当前标题作为块头
        for j in range(0, len(text), chunk_size - overlap):
            chunk = text[j:j+chunk_size]
            chunks.append(f"[CCH] {current_header}\n{chunk}")  # 格式：块头+内容
    
    return chunks
```  
- **设计要点**：  
  - 每个块以`[CCH] 标题`作为前缀，明确标识上下文来源；  
  - 当内容跨章节时，优先使用最近的标题作为块头（如段落同时属于章和节时，使用节标题）。  


##### 3. 嵌入生成与语义检索  
```python
# 嵌入生成（与传统RAG类似，但输入为带CCH的块）
def create_cch_embeddings(chunks, model="text-embedding-ada-002"):
    """生成带上下文头的块嵌入"""
    return client.embeddings.create(model=model, input=chunks)

# 检索时，CCH块的嵌入包含标题语义，提升相关性
def cch_semantic_search(query, chunks, embeddings, k=5):
    """基于CCH块的语义检索"""
    # 计算查询与块嵌入的相似度（与传统RAG相同）
    # 但因块头存在，相似度过滤更精准
    return top_k_relevant_chunks
```  
- **技术优势**：  
  块头中的关键词（如“人工智能”“核心技术”）会提升相关查询的相似度得分，使检索结果更聚焦于主题相关的块。  


##### 4. 响应生成与评估  
```python
# 生成响应时，块头信息帮助LLM理解内容的上下文归属
system_prompt = "你是AI助手，基于提供的带标题上下文回答问题。标题用于说明内容主题，回答需结合标题与内容。"

# 评估时，CCH技术的得分通常更高，因回答更符合文档结构
def evaluate_cch_response(ai_answer, true_answer, context_chunks):
    """评估带CCH的响应准确性"""
    # 检查回答是否利用了块头中的主题信息
    # 例如：回答“AI核心技术”时，是否引用了标题为“第三章 人工智能核心技术”的块
    return accuracy_score
```  


#### 四、CCH技术与传统分块的对比  
| 维度               | 传统分块                          | CCH分块                          |
|--------------------|-----------------------------------|-----------------------------------|
| **块结构**         | 纯内容（如“机器学习是AI的基础...”） | 标题+内容（如“第三章 AI核心技术\n机器学习是AI的基础...”） |
| **语义完整性**     | 缺乏上下文归属，可能跨主题混淆    | 明确主题归属，减少跨章节误匹配    |
| **检索准确率**     | 仅基于内容关键词匹配              | 结合标题主题和内容关键词匹配      |
| **回答相关性**     | 可能生成脱离文档结构的回答        | 回答更符合文档章节逻辑            |
| **实现成本**       | 低（无需解析文档结构）            | 中（需解析标题层级）              |  


#### 五、CCH技术的适用场景  
1. **学术文献问答**：  
   - 块头为“第四章 实验方法”的内容块，在用户查询“该研究使用了哪些实验方法”时被优先检索。  

2. **企业文档检索**：  
   - 块头为“政策-报销流程-差旅”的文本块，在用户查询“差旅报销需要哪些凭证”时精准匹配。  

3. **多文档RAG系统**：  
   - 不同文档的块头可包含文档名称（如“2024年财报.pdf-第二季度营收”），避免跨文档信息混淆。  


#### 六、CCH技术的扩展与优化  
1. **动态块头生成**：  
   - 对无明确标题的文档，使用LLM生成摘要式块头（如“关于机器学习分类方法的介绍”）。  

2. **分层块头策略**：  
   - 根据标题层级（章→节→子节）赋予不同权重，例如章标题影响全局主题，子节标题影响局部语义。  

3. **块头压缩技术**：  
   - 对过长标题（如“第三部分 人工智能在金融领域的应用案例分析-银行智能风控系统”），使用LLM生成精简版块头（如“金融领域AI应用-银行风控”），减少嵌入向量中的冗余信息。  


#### 七、潜在挑战与解决方案  
1. **标题解析误差**：  
   - 非结构化文档（如纯文本）的标题识别可能出错（如误将普通段落识别为标题）。  
   - **解决方案**：结合正则表达式和LLM分类，提升标题识别准确率。  

2. **块头长度控制**：  
   - 过长块头（如500字标题）会占用嵌入向量的语义空间，降低内容表征精度。  
   - **解决方案**：限制块头长度（如不超过100字），或使用标题嵌入与内容嵌入的融合策略。  

3. **多语言支持**：  
   - 跨语言文档的标题解析（如中文标题→英文内容）可能导致语义偏差。  
   - **解决方案**：使用多语言嵌入模型（如XLM-R），或对标题和内容分别进行语言适配处理。  

CCH技术通过显式保留文档结构信息，在不显著增加计算成本的前提下，有效提升了RAG系统的检索质量和回答相关性，尤其适合处理具有明确层级结构的专业文档。

## Setting Up the Environment
We begin by importing necessary libraries.

In [2]:
import os
import numpy as np
import json
from openai import OpenAI
import fitz
from tqdm import tqdm

In [1]:
pip install PymuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.8 MB/s eta 0:00:00


## Extracting Text and Identifying Section Headers
We extract text from a PDF while also identifying section titles (potential headers for chunks).

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file and prints the first `num_chars` characters.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    # Open the PDF file
    mypdf = fitz.open(pdf_path)
    all_text = ""  # Initialize an empty string to store the extracted text

    # Iterate through each page in the PDF
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # Get the page
        text = page.get_text("text")  # Extract text from the page
        all_text += text  # Append the extracted text to the all_text string

    return all_text  # Return the extracted text

## Setting Up the OpenAI API Client
We initialize the OpenAI client to generate embeddings and responses.

In [4]:
# Initialize the OpenAI client with the base URL and API key
client = OpenAI(
    base_url="http://xxx/v1/",
    api_key="xxxxxxt9"  # Retrieve the API key from environment variables
)

## Chunking Text with Contextual Headers
To improve retrieval, we generate descriptive headers for each chunk using an LLM model.

In [8]:
def generate_chunk_header(chunk, model="gpt-4.1"):
    """
    Generates a title/header for a given text chunk using an LLM.

    Args:
    chunk (str): The text chunk to summarize as a header.
    model (str): The model to be used for generating the header. Default is "meta-llama/Llama-3.2-3B-Instruct".

    Returns:
    str: Generated header/title.
    """
    # Define the system prompt to guide the AI's behavior
    system_prompt = "Generate a concise and informative title for the given text."

    # Generate a response from the AI model based on the system prompt and text chunk
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chunk}
        ]
    )

    # Return the generated header/title, stripping any leading/trailing whitespace
    return response.choices[0].message.content.strip()

### 带智能标题的文本分块函数解析与优化

这个`chunk_text_with_headers`函数实现了一个关键功能：**自动为每个文本块生成描述性标题**，这是上下文增强检索(CCH)的核心环节。下面从功能设计到性能优化进行全面解析：

#### 一、函数核心逻辑与流程
```python
def chunk_text_with_headers(text, n, overlap):
    """
    将文本分块并为每个块生成描述性标题
    
    参数:
    text (str): 待分块的完整文本
    n (int): 每个块的字符长度
    overlap (int): 相邻块的重叠字符数
    
    返回:
    List[dict]: 包含{标题, 文本内容}的字典列表
    """
    chunks = []
    
    # 1. 按固定步长滑动窗口分块
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # 提取文本块
        
        # 2. 调用LLM生成标题(关键步骤)
        header = generate_chunk_header(chunk)
        
        # 3. 存储带标题的文本块
        chunks.append({"header": header, "text": chunk})
    
    return chunks
```

#### 二、标题生成函数`generate_chunk_header`实现
虽然代码中未定义该函数，但从调用可以推断其功能：
```python
def generate_chunk_header(chunk_text):
    """使用LLM为文本块生成描述性标题"""
    # 构建提示词
    prompt = f"""
    为以下文本生成一个简洁的标题(不超过15个词)，准确概括其核心内容：
    {chunk_text[:500]}  # 截断过长文本，避免超出token限制
    
    标题必须：
    1. 反映文本的核心主题
    2. 不包含多余修饰词
    3. 格式为短语而非完整句子
    
    示例输入: "机器学习是人工智能的一个分支，它使用算法..."
    示例输出: "机器学习的定义与应用"
    """
    
    # 调用LLM生成标题
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0.2,  # 低随机性，保证标题确定性
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content.strip()
```

#### 三、关键技术点分析
1. **滑动窗口分块策略**：
   - 步长计算：`n - overlap`（如n=1000, overlap=200时，每800字符生成一个新块）
   - 边界处理：自动处理文本末尾不足n的情况（如最后一块可能只有500字符）

2. **标题生成的LLM参数调优**：
   - 模型选择：建议使用`gpt-3.5-turbo`（成本低，速度快）
   - temperature=0.2：降低随机性，生成更确定性的标题
   - 提示词设计：通过明确约束（如"不超过15个词"）控制输出格式

3. **性能优化考虑**：
   - 批处理：每10-20个块批量调用一次LLM，减少API请求次数
   - 缓存机制：相同文本块复用已生成的标题（如使用LRU缓存）

#### 四、完整处理流程示例
```python
# 1. 从PDF提取文本
pdf_path = "AI_Information.pdf"
extracted_text = extract_text_from_pdf(pdf_path)

# 2. 分块并生成标题
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# 3. 生成嵌入向量
embeddings = create_embeddings([
    f"[标题] {chunk['header']}\n[内容] {chunk['text']}"
    for chunk in text_chunks
])

# 4. 示例：打印第一个块
print("示例标题:", text_chunks[0]['header'])
print("示例内容:", text_chunks[0]['text'][:100], "...")
```

#### 五、输出示例与效果对比
**原始文本块**（前100字符）：
```
"人工智能(AI)是计算机科学的一个分支，旨在使机器能够模拟人类智能。AI系统通过学习数据模式来执行任务，不需要明确的编程指令..."
```

**生成的标题**：
```
"人工智能的定义与核心原理"
```

**对比传统分块**：
| 方法               | 检索"AI定义"的相关性 | 上下文连贯性 |
|--------------------|----------------------|--------------|
| 无标题分块         | 中等（需匹配关键词） | 低           |
| 带智能标题分块     | 高（标题直接匹配）   | 高           |

#### 六、性能优化方案
1. **并行处理**：
```python
# 使用concurrent.futures实现并行标题生成
from concurrent.futures import ThreadPoolExecutor

def process_chunk(chunk):
    header = generate_chunk_header(chunk)
    return {"header": header, "text": chunk}

with ThreadPoolExecutor(max_workers=5) as executor:
    text_chunks = list(executor.map(process_chunk, chunks))
```

2. **标题质量提升**：
```python
# 添加标题质量验证逻辑
def validate_header(header, chunk_text):
    # 计算标题与文本的语义相似度
    header_embedding = create_embeddings(header)
    chunk_embedding = create_embeddings(chunk_text[:500])
    
    similarity = cosine_similarity(header_embedding, chunk_embedding)
    return similarity > 0.6  # 设置相似度阈值

# 对质量不达标的标题进行二次生成
if not validate_header(header, chunk_text):
    header = generate_chunk_header(chunk_text, retry=True)
```

#### 七、潜在问题与解决方案
1. **标题生成失败**：
   - 捕获API异常，设置默认标题（如"未分类内容"）
   - 添加重试机制（如使用`tenacity`库）

2. **标题重复问题**：
   - 维护已生成标题集合，发现重复时添加编号（如"机器学习基础-1"）
   - 调整提示词，要求标题具有唯一性

3. **成本控制**：
   - 对短文本块（如<300字符）跳过标题生成，使用前20个字符作为标题
   - 实现标题生成的开关参数，允许在非关键场景关闭该功能

通过这种带智能标题的分块方法，RAG系统的检索准确率通常可提升20-30%，尤其在处理长文档时效果显著。

In [6]:
def chunk_text_with_headers(text, n, overlap):
    """
    Chunks text into smaller segments and generates headers.

    Args:
    text (str): The full text to be chunked.
    n (int): The chunk size in characters.
    overlap (int): Overlapping characters between chunks.

    Returns:
    List[dict]: A list of dictionaries with 'header' and 'text' keys.
    """
    chunks = []  # Initialize an empty list to store chunks

    # Iterate through the text with the specified chunk size and overlap
    for i in range(0, len(text), n - overlap):
        chunk = text[i:i + n]  # Extract a chunk of text
        header = generate_chunk_header(chunk)  # Generate a header for the chunk using LLM
        chunks.append({"header": header, "text": chunk})  # Append the header and chunk to the list

    return chunks  # Return the list of chunks with headers

## Extracting and Chunking Text from a PDF File
Now, we load the PDF, extract text, and split it into chunks.

In [9]:
# Define the PDF file path
pdf_path = "AI_Information.pdf"

# Extract text from the PDF file
extracted_text = extract_text_from_pdf(pdf_path)

# Chunk the extracted text with headers
# We use a chunk size of 1000 characters and an overlap of 200 characters
text_chunks = chunk_text_with_headers(extracted_text, 1000, 200)

# Print a sample chunk with its generated header
print("Sample Chunk:")
print("Header:", text_chunks[0]['header'])
print("Content:", text_chunks[0]['text'])

Sample Chunk:
Header: Introduction and Historical Overview of Artificial Intelligence
Content: Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. 

## Creating Embeddings for Headers and Text
We create embeddings for both headers and text to improve retrieval accuracy.

In [10]:
def create_embeddings(text, model="text-embedding-ada-002"):
    """
    Creates embeddings for the given text.

    Args:
    text (str): The input text to be embedded.
    model (str): The embedding model to be used. Default is "BAAI/bge-en-icl".

    Returns:
    dict: The response containing the embedding for the input text.
    """
    # Create embeddings using the specified model and input text
    response = client.embeddings.create(
        model=model,
        input=text
    )
    # Return the embedding from the response
    return response.data[0].embedding

In [11]:
# Generate embeddings for each chunk
embeddings = []  # Initialize an empty list to store embeddings

# Iterate through each text chunk with a progress bar
for chunk in tqdm(text_chunks, desc="Generating embeddings"):
    # Create an embedding for the chunk's text
    text_embedding = create_embeddings(chunk["text"])
    # Create an embedding for the chunk's header
    header_embedding = create_embeddings(chunk["header"])
    # Append the chunk's header, text, and their embeddings to the list
    embeddings.append({"header": chunk["header"], "text": chunk["text"], "embedding": text_embedding, "header_embedding": header_embedding})

Generating embeddings: 100%|██████████| 42/42 [02:08<00:00,  3.06s/it]


## Performing Semantic Search
We implement cosine similarity to find the most relevant text chunks for a user query.

In [12]:
def cosine_similarity(vec1, vec2):
    """
    Computes cosine similarity between two vectors.

    Args:
    vec1 (np.ndarray): First vector.
    vec2 (np.ndarray): Second vector.

    Returns:
    float: Cosine similarity score.
    """
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [13]:
def semantic_search(query, chunks, k=5):
    """
    Searches for the most relevant chunks based on a query.

    Args:
    query (str): User query.
    chunks (List[dict]): List of text chunks with embeddings.
    k (int): Number of top results.

    Returns:
    List[dict]: Top-k most relevant chunks.
    """
    # Create an embedding for the query
    query_embedding = create_embeddings(query)

    similarities = []  # Initialize a list to store similarity scores

    # Iterate through each chunk to calculate similarity scores
    for chunk in chunks:
        # Compute cosine similarity between query embedding and chunk text embedding
        sim_text = cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"]))
        # Compute cosine similarity between query embedding and chunk header embedding
        sim_header = cosine_similarity(np.array(query_embedding), np.array(chunk["header_embedding"]))
        # Calculate the average similarity score
        avg_similarity = (sim_text + sim_header) / 2
        # Append the chunk and its average similarity score to the list
        similarities.append((chunk, avg_similarity))

    # Sort the chunks based on similarity scores in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)
    # Return the top-k most relevant chunks
    return [x[0] for x in similarities[:k]]

### 语义搜索函数解析：基于标题和内容双重匹配的混合检索

这个`semantic_search`函数实现了一个关键优化：**同时利用文本块内容和标题的嵌入向量进行检索**，显著提升了语义匹配的准确性。下面从技术实现到性能优化进行详细解析：

#### 一、核心算法逻辑
```python
def semantic_search(query, chunks, k=5):
    """
    基于语义相似度检索最相关的文本块
    
    参数:
    query (str): 用户查询
    chunks (List[dict]): 包含文本块和嵌入向量的字典列表
    k (int): 返回的结果数量
    
    返回:
    List[dict]: 最相关的k个文本块
    """
    # 1. 生成查询的嵌入向量
    query_embedding = create_embeddings(query)
    
    similarities = []
    
    # 2. 计算每个文本块的匹配度
    for chunk in chunks:
        # 2.1 内容相似度: 查询与文本内容的匹配度
        sim_text = cosine_similarity(
            np.array(query_embedding),
            np.array(chunk["embedding"])
        )
        
        # 2.2 标题相似度: 查询与文本标题的匹配度
        sim_header = cosine_similarity(
            np.array(query_embedding),
            np.array(chunk["header_embedding"])
        )
        
        # 2.3 加权平均: 标题相似度权重可调整
        avg_similarity = (sim_text + sim_header) / 2
        
        similarities.append((chunk, avg_similarity))
    
    # 3. 按相似度排序并返回前k个结果
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in similarities[:k]]
```

#### 二、算法设计亮点
1. **混合相似度计算**：
   - 内容相似度：捕获查询与文本细节的匹配
   - 标题相似度：捕获查询与文本主题的匹配
   - 平均策略：平衡主题相关性和细节匹配度

2. **向量计算优化**：
   - 使用NumPy数组加速余弦相似度计算
   - 可扩展为批量向量计算（适用于大规模数据集）

3. **数据结构设计**：
   - 每个文本块需预计算两个嵌入向量：
     ```python
     {
         "header": "人工智能的定义",
         "text": "人工智能是...",
         "embedding": [...],         # 文本内容的嵌入向量
         "header_embedding": [...]   # 标题的嵌入向量
     }
     ```

#### 三、性能优化方案
1. **索引加速**：
```python
# 使用FAISS库构建向量索引
import faiss

def build_faiss_index(chunks):
    """构建FAISS索引加速语义搜索"""
    # 合并内容和标题的嵌入向量（可调整权重）
    dim = len(chunks[0]["embedding"])
    index = faiss.IndexFlatIP(dim)  # 内积索引，适合余弦相似度
    
    # 构建索引
    all_vectors = []
    for chunk in chunks:
        # 可调整标题和内容的权重
        vector = 0.6 * np.array(chunk["embedding"]) + 0.4 * np.array(chunk["header_embedding"])
        all_vectors.append(vector.astype('float32'))
    
    index.add(np.vstack(all_vectors))
    return index

# 优化后的搜索函数
def optimized_semantic_search(query, chunks, index, k=5):
    query_vector = create_embeddings(query)
    query_vector = (0.6 * np.array(query_vector) + 0.4 * np.array(query_vector)).astype('float32')
    
    # 使用FAISS搜索
    distances, indices = index.search(np.array([query_vector]), k)
    
    # 返回最相似的文本块
    return [chunks[i] for i in indices[0]]
```

2. **缓存机制**：
```python
# 使用LRU缓存避免重复计算
from functools import lru_cache

@lru_cache(maxsize=1000)  # 缓存最近1000次查询结果
def cached_semantic_search(query, chunks, k=5):
    return semantic_search(query, chunks, k)
```

#### 四、检索效果对比
| 查询示例 | 传统检索（仅内容） | 混合检索（内容+标题） |
|----------|--------------------|----------------------|
| "AI的定义" | 可能返回包含"AI"但主题不相关的段落 | 优先返回标题为"人工智能的定义"的段落 |
| "机器学习应用" | 需精确匹配"应用"关键词 | 标题包含"应用"的段落权重更高 |

#### 五、扩展应用场景
1. **多模态检索**：
   - 对图片/表格添加文本标题，参与语义检索
   - 示例：
     ```python
     {
         "header": "图3: 2020-2023年AI专利数量增长趋势",
         "text": "该图展示了...",
         "embedding": [...],
         "header_embedding": [...]
     }
     ```

2. **跨语言检索**：
   - 使用多语言嵌入模型（如XLM-R）
   - 支持查询语言与文档语言不一致的场景

3. **知识图谱增强**：
   - 将标题映射到知识图谱实体
   - 示例：标题"Transformer架构"关联到实体`https://dbpedia.org/resource/Transformer_(machine_learning_model)`

#### 六、参数调优建议
1. **相似度权重调整**：
   ```python
   # 为标题相似度分配更高权重（适用于主题优先的场景）
   avg_similarity = 0.3 * sim_text + 0.7 * sim_header
   ```

2. **嵌入模型选择**：
   - 通用场景：text-embedding-ada-002
   - 专业领域：BAAI/bge-large-en（训练时加入领域语料）

3. **k值选择**：
   - 对话系统：k=3（快速响应）
   - 研究助手：k=10（提供更多上下文）

通过这种基于标题和内容双重匹配的语义搜索方法，RAG系统的检索准确率通常可提升20-30%，尤其在处理长文档和专业领域内容时效果显著。

## Running a Query on Extracted Chunks

In [14]:
# Load validation data
with open('val.json') as f:
    data = json.load(f)

query = data[0]['question']

# Retrieve the top 2 most relevant text chunks
top_chunks = semantic_search(query, embeddings, k=2)

# Print the results
print("Query:", query)
for i, chunk in enumerate(top_chunks):
    print(f"Header {i+1}: {chunk['header']}")
    print(f"Content:\n{chunk['text']}\n")

Query: What is 'Explainable AI' and why is it considered important?
Header 1: Key Principles for Building Trustworthy AI: Explainability, Privacy, Accountability, and Transparency
Content:
systems. Explainable AI (XAI) 
techniques aim to make AI decisions more understandable, enabling users to assess their 
fairness and accuracy. 
Privacy and Data Protection 
AI systems often rely on large amounts of data, raising concerns about privacy and data 
protection. Ensuring responsible data handling, implementing privacy-preserving techniques, 
and complying with data protection regulations are crucial. 
Accountability and Responsibility 
Establishing accountability and responsibility for AI systems is essential for addressing potential 
harms and ensuring ethical behavior. This includes defining roles and responsibilities for 
developers, deployers, and users of AI systems. 
Chapter 20: Building Trust in AI 
Transparency and Explainability 
Transparency and explainability are key to building

## Generating a Response Based on Retrieved Chunks

In [16]:
# Define the system prompt for the AI assistant
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="gpt-4.1"):
    """
    Generates a response from the AI model based on the system prompt and user message.

    Args:
    system_prompt (str): The system prompt to guide the AI's behavior.
    user_message (str): The user's message or query.
    model (str): The model to be used for generating the response. Default is "meta-llama/Llama-2-7B-chat-hf".

    Returns:
    dict: The response from the AI model.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# Create the user prompt based on the top chunks
user_prompt = "\n".join([f"Header: {chunk['header']}\nContent:\n{chunk['text']}" for chunk in top_chunks])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# Generate AI response
ai_response = generate_response(system_prompt, user_prompt)

In [17]:
print(ai_response)

ChatCompletion(id='chatcmpl-BiJxm4X5q57sWmodqQEkcK5ryALzn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Explainable AI (XAI) techniques aim to make AI decisions more understandable, enabling users to assess their fairness and accuracy. It is considered important because making AI systems understandable and providing insights into their decision-making processes helps users assess their reliability and fairness, which is key to building trust in AI. Additionally, enhancing transparency and explainability is crucial for building trust and accountability, especially since many AI systems, particularly deep learning models, are "black boxes" that are difficult to interpret.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text

## Evaluating the AI Response
We compare the AI response with the expected answer and assign a score.

In [18]:
# Define evaluation system prompt
evaluate_system_prompt = """You are an intelligent evaluation system.
Assess the AI assistant's response based on the provided context.
- Assign a score of 1 if the response is very close to the true answer.
- Assign a score of 0.5 if the response is partially correct.
- Assign a score of 0 if the response is incorrect.
Return only the score (0, 0.5, or 1)."""

# Extract the ground truth answer from validation data
true_answer = data[0]['ideal_answer']

# Construct evaluation prompt
evaluation_prompt = f"""
User Query: {query}
AI Response: {ai_response}
True Answer: {true_answer}
{evaluate_system_prompt}
"""

# Generate evaluation score
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# Print the evaluation score
print("Evaluation Score:", evaluation_response.choices[0].message.content)

Evaluation Score: 1


In [19]:
print(evaluation_response)

ChatCompletion(id='chatcmpl-BiJyIbz3BLiXLTsPZwizopbFf2OVu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1749902902, model='gpt-4.1-2025-04-14', object='chat.completion', service_tier=None, system_fingerprint='fp_07e970ab25', usage=CompletionUsage(completion_tokens=2, prompt_tokens=719, total_tokens=721, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens

### ChatCompletion响应参数详解：从结构到含义的全面解析  

以下是对ChatCompletion响应中各参数的详细拆解，帮助理解LLM交互的技术细节：  


#### 一、顶级字段：响应元数据  
```python
ChatCompletion(
    id='chatcmpl-BiJyIbz3BLiXLTsPZwizopbFf2OVu',  # ①
    created=1749902902,  # ②
    model='gpt-4.1-2025-04-14',  # ③
    object='chat.completion',  # ④
    service_tier=None,
    system_fingerprint='fp_07e970ab25',
    # 其他字段...
)
```  

1. **`id`**：  
   - **类型**：字符串  
   - **含义**：本次API调用的唯一标识符，用于追踪请求记录（如调试、计费查询）。  
   - **示例**：`chatcmpl-BiJyIbz3BLiXLTsPZwizopbFf2OVu`（以`chatcmpl-`开头）。  

2. **`created`**：  
   - **类型**：时间戳（UTC秒）  
   - **含义**：响应生成的时间，可用于计算请求耗时。  
   - **转换示例**：`1749902902` → `2025年1月23日 14:48:22`（需通过时间转换工具解析）。  

3. **`model`**：  
   - **类型**：字符串  
   - **含义**：生成响应的模型版本。  
   - **示例**：`gpt-4.1-2025-04-14`（表示2025年4月14日发布的GPT-4.1版本）。  

4. **`object`**：  
   - **类型**：字符串  
   - **含义**：响应对象的类型，固定为`chat.completion`（表示聊天完成响应）。  


#### 二、核心字段：choices（响应内容）  
```python
choices=[
    Choice(
        finish_reason='stop',  # ①
        index=0,  # ②
        logprobs=None,
        message=ChatCompletionMessage(  # ③
            content='1',
            role='assistant',
            # 其他message字段...
        ),
        # 其他Choice字段...
    )
]
```  

1. **`finish_reason`**：  
   - **类型**：字符串  
   - **含义**：模型停止生成的原因，常见值：  
     - `stop`：正常生成完毕（遇到结束符或达到最大token数）；  
     - `length`：因达到`max_tokens`限制而中断；  
     - `function_call`：因调用工具函数而停止（适用于函数调用场景）。  

2. **`index`**：  
   - **类型**：整数  
   - **含义**：响应在多选项中的索引（通常`index=0`，因默认只返回1个结果）。  

3. **`message`**：  
   - **类型**：ChatCompletionMessage对象  
   - **核心子字段**：  
     - `role`：角色标识，固定为`assistant`（表示助手响应）；  
     - `content`：生成的文本内容（如`'1'`）；  
     - `function_call`：若调用了函数，此处包含函数名和参数（本例中为`None`）。  


#### 三、内容过滤字段：安全审核结果  
```python
# 在choices和prompt_filter_results中均有安全过滤字段
{
    'hate': {'filtered': False, 'severity': 'safe'},  # 仇恨内容检测
    'self_harm': {'filtered': False, 'severity': 'safe'},  # 自残内容检测
    'sexual': {'filtered': False, 'severity': 'safe'},  # 色情内容检测
    'violence': {'filtered': False, 'severity': 'safe'},  # 暴力内容检测
    # 其他过滤类型...
}
```  

- **字段含义**：  
  - `filtered`：是否触发过滤（`True`表示内容被屏蔽，`False`表示通过）；  
  - `severity`：风险等级（如`'safe'`表示安全，`'high'`表示高风险）。  
- **作用**：OpenAI的内容安全机制，自动检测并过滤违规内容。  


#### 四、用量统计：usage  
```python
usage=CompletionUsage(
    completion_tokens=2,  # ①
    prompt_tokens=719,  # ②
    total_tokens=721,  # ③
    # 其他用量细节...
)
```  

1. **`completion_tokens`**：  
   - **含义**：生成响应消耗的token数（本例中为2，即输出`'1'`约2个token）。  

2. **`prompt_tokens`**：  
   - **含义**：输入提示词消耗的token数（如用户问题、上下文等，本例中为719）。  

3. **`total_tokens`**：  
   - **含义**：总消耗token数（`prompt_tokens + completion_tokens`，本例中719+2=721）。  
- **重要性**：token用量直接影响API计费（如GPT-4.1每1k tokens约0.03美元）。  


#### 五、扩展字段：prompt_filter_results  
```python
prompt_filter_results=[
    {
        'prompt_index': 0,
        'content_filter_results': {
            'jailbreak': {'filtered': False, 'detected': False},  # 越狱攻击检测
            # 其他过滤类型与choices中的字段一致...
        }
    }
]
```  

- **作用**：对输入提示词进行安全检测，防止用户通过prompt注入（如诱导模型绕过安全限制）。  
- **`jailbreak`字段**：检测是否存在“越狱”尝试（即诱导模型生成违规内容），`detected=False`表示未检测到风险。  


#### 六、参数应用场景与实践价值  
1. **调试与追踪**：  
   - 通过`id`和`created`字段关联请求与响应，排查超时或异常问题。  

2. **成本控制**：  
   - 监控`usage.total_tokens`统计API消耗，优化提示词长度以降低成本（如精简上下文）。  

3. **安全合规**：  
   - 检查`content_filter_results`确保输出无违规内容，尤其在医疗、金融等敏感领域。  

4. **模型版本管理**：  
   - 通过`model`字段确认当前使用的模型版本，避免因版本更新导致的效果波动（如API默认升级时）。  


#### 七、典型异常场景处理  
1. **`finish_reason='length'`**：  
   - **处理**：增加`max_tokens`参数值，或精简输入提示词以允许更多输出。  

2. **`content_filter_results['filtered']=True`**：  
   - **处理**：修改提示词以消除违规内容，或联系OpenAI调整安全策略（如企业级服务可申请定制过滤规则）。  

3. **`usage.total_tokens`异常高**：  
   - **优化**：  
     - 使用token计数器（如`tiktoken`库）提前估算提示词token数；  
     - 对长文本进行分块处理，避免一次性传入过多内容。  


#### 八、字段解析代码示例  
```python
# 解析响应参数的Python代码示例
response = client.chat.completions.create(...)  # 假设已获取响应

# 提取核心信息
print(f"模型版本: {response.model}")
print(f"响应内容: {response.choices[0].message.content}")
print(f"总消耗token: {response.usage.total_tokens}")

# 安全检测结果
if any(
    filter_result['filtered']
    for choice in response.choices
    for filter_result in choice.content_filter_results.values()
):
    print("警告：响应包含被过滤的内容")
```  

通过理解这些参数，开发者可更精准地控制LLM交互过程，优化成本、安全性和响应质量。

### RAG系统评估模块详解：自动化评分机制与实现原理

这段代码实现了RAG系统中至关重要的**自动化评估模块**，通过LLM自身对生成的回答进行评分。这种方法巧妙地利用了模型的理解能力，避免了传统基于规则或简单关键词匹配的评估局限性。

#### 一、评估系统的核心设计
```python
# 定义评估系统的提示词
evaluate_system_prompt = """You are an intelligent evaluation system.
Assess the AI assistant's response based on the provided context.
- Assign a score of 1 if the response is very close to the true answer.
- Assign a score of 0.5 if the response is partially correct.
- Assign a score of 0 if the response is incorrect.
Return only the score (0, 0.5, or 1)."""
```

#### 关键设计点：
1. **角色定位**：将评估者定义为"intelligent evaluation system"，强调其专业评估能力
2. **评分标准**：
   - 1分：回答与标准答案高度一致
   - 0.5分：部分正确（如遗漏次要信息但包含关键内容）
   - 0分：回答错误或完全脱离上下文
3. **输出约束**：强制返回数值评分（避免生成解释性文本）

#### 二、评估流程实现
```python
# 构建评估提示词
evaluation_prompt = f"""
User Query: {query}
AI Response: {ai_response}
True Answer: {true_answer}
{evaluate_system_prompt}
"""

# 生成评估分数
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 提取评分结果
print("Evaluation Score:", evaluation_response.choices[0].message.content)
```

#### 流程解析：
1. **信息输入**：
   - 用户原始问题（User Query）
   - AI生成的回答（AI Response）
   - 标准答案（True Answer）
   - 评估规则（evaluate_system_prompt）

2. **评估执行**：
   - 调用相同的`generate_response`函数，但使用评估专用的提示词
   - 模型需根据上述信息输出0、0.5或1

3. **结果输出**：
   - 直接获取模型返回的数值作为评分结果

#### 三、评分机制的工作原理
评估模型会执行以下分析步骤：
1. **相关性检查**：AI回答是否针对用户问题
2. **事实核对**：
   - 关键事实是否与标准答案一致
   - 是否包含上下文未提供的信息（即产生幻觉）
3. **完整性评估**：
   - 是否涵盖标准答案的所有要点
   - 是否遗漏关键细节

#### 四、评估系统的优势
1. **语义理解能力**：
   - 能识别同义表达（如"AI"与"人工智能"视为等同）
   - 理解复杂逻辑关系（如因果、对比）

2. **上下文感知**：
   - 区分回答是基于上下文还是模型自身知识
   - 严格执行"仅根据上下文回答"的约束

3. **适应性强**：
   - 可评估不同类型的问题（如事实性、解释性）
   - 无需为特定领域定制评估规则

#### 五、潜在问题与优化方向
1. **评分一致性**：
   - 同一回答可能因评估时的随机性得到不同分数
   - **优化**：设置temperature=0，或多次评估取平均值

2. **标准答案偏差**：
   - 人工标注的标准答案可能存在主观性
   - **优化**：采用多标注者共识，或使用金标准数据集

3. **评估成本**：
   - 每次评估需额外调用一次LLM
   - **优化**：
     - 批量评估多个回答
     - 使用轻量级模型进行初步评估

#### 六、评估结果的应用场景
1. **模型选择与调优**：
   - 比较不同LLM在特定任务上的表现
   - 优化检索参数（如k值、上下文大小）

2. **系统监控**：
   - 实时监测RAG系统的回答质量
   - 识别性能下降的时间点并触发警报

3. **迭代改进**：
   - 分析低评分回答，针对性改进：
     - 增强检索相关性
     - 优化提示工程
     - 改进分块策略

#### 七、扩展实现：更精细的评估指标
除了整体评分外，还可扩展评估维度：
```python
# 扩展评估系统，返回多维度评分
expanded_evaluate_prompt = """
Assess the AI response based on the context:
1. Relevance (0-1): How relevant is the response to the query?
2. Factuality (0-1): How factual is the response?
3. Completeness (0-1): How complete is the response?
4. Coherence (0-1): How coherent is the response?

Return scores in JSON format:
{
"relevance": 0.8,
"factuality": 1.0,
"completeness": 0.6,
"coherence": 0.9
}
"""

# 解析多维评分结果
scores = json.loads(evaluation_response.choices[0].message.content)
print(f"事实性得分: {scores['factuality']}")
print(f"完整性得分: {scores['completeness']}")
```

这种多维评估能更精确地定位系统弱点，指导针对性优化。

#### 八、最佳实践建议
1. **评估数据准备**：
   - 使用领域内代表性问题
   - 每个问题配备高质量标准答案
   - 包含多样化的问题类型（事实性、开放性、对比性）

2. **评估模型选择**：
   - 使用比生成模型更强大的LLM进行评估（如用GPT-4评估GPT-3.5的回答）

3. **结果可视化**：
   - 定期生成评估报告，跟踪系统性能变化
   - 可视化不同维度的评分趋势

通过这种自动化评估机制，RAG系统能够持续优化，逐步提升回答质量和用户满意度。